# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [5]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import  BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.linear_model import LogisticRegression

## Read the CSV and Perform Basic Data Cleaning

In [6]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [7]:
# Create our features
X = pd.get_dummies(df.drop('loan_status', axis=1))


In [42]:
# Create our target
y = df.loan_status.to_frame("loan_status")

In [43]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [44]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [45]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state =1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [46]:
# Create the StandardScaler instance
scaler =  StandardScaler()

In [47]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)


In [48]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [49]:
# Resample the training data with the BalancedRandomForestClassifier
bal_rf_model = BalancedRandomForestClassifier(n_estimators=700, max_depth=15,  random_state=50)
bal_rf_model.fit(X_train_scaled, y_train)


BalancedRandomForestClassifier(max_depth=15, n_estimators=700, random_state=50)

In [52]:
y_bal_rf_pred = bal_rf_model.predict(X_test_scaled)
bal_acc_score = balanced_accuracy_score(y_test, y_bal_rf_pred)
print(f" \033[1m Balance Accuracy Score for Balance Random Forest Classifier {bal_acc_score} \033[0m")

  Balance Accuracy Score for Balance Random Forest Classifier 0.8140342289423113 


In [ ]:
# Calculated the balanced accuracy score
y_balrf_pred = bal_rf_model.predict(X_test_scaled)
bal_acc_score = balanced_accuracy_score(y_test, y_balrf_pred)
print(f"Balance Accuracy Score for Balanced Random Forest Classifier {bal_acc_score}")

In [53]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_bal_rf_pred)
cm_df = pd.DataFrame(cm , index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])
print("Confusion Matrix")
display(cm_df) 

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,61,26
Actual 1,1251,15867


In [67]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_bal_rf_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.70      0.93      0.09      0.81      0.64        87
   low_risk       1.00      0.93      0.70      0.96      0.81      0.66     17118

avg / total       0.99      0.93      0.70      0.96      0.81      0.66     17205



In [62]:
# List the features sorted in descending order by feature importance
 # Random Forests in sklearn will automatically calculate feature importance
importances = bal_rf_model.feature_importances_

# We can sort the features by their importance

importances_df = pd.DataFrame(importances, index = X_train.columns, columns=['Feature Importance']).sort_values('Feature Importance', ascending=False)

importances_df.head()


,Feature Importance
total_rec_prncp,0.071484
total_pymnt,0.060788
total_rec_int,0.059011
last_pymnt_amnt,0.058050
total_pymnt_inv,0.056972


### Easy Ensemble Classifier

In [57]:
# Train the Classifier
ee_model20 = EasyEnsembleClassifier(n_estimators=100, random_state=1)

ee_model20.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [63]:
# Calculated the balanced accuracy score
y_ee_pred = ee_model20.predict(X_test_scaled)
ee_acc_score = balanced_accuracy_score(y_test, y_ee_pred)
print(f" \033[1m Easy Ensemble Classifier Balanced Accuracy Score {ee_acc_score}\033[0m")

  Easy Ensemble Classifier Balanced Accuracy Score 0.9254565671948463


In [64]:
# Display the confusion matrix
cm = confusion_matrix(y_ee_pred, y_test)
cm_df = pd.DataFrame(cm, index = ["Actual 0", "Actual 1"], columns=["Perdicted 0","Perdicted 1"])
cm_df

,Perdicted 0,Perdicted 1
Actual 0,79,978
Actual 1,8,16140


In [65]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_ee_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



In [66]:
print(f" Balance Accuracy Score for Balance Random Forest Classifier {bal_acc_score}")
print(f"Easy Ensemble Classifier Balanced Accuracy Score {ee_acc_score}")

 Balance Accuracy Score for Balance Random Forest Classifier 0.8140342289423113
Easy Ensemble Classifier Balanced Accuracy Score 0.9254565671948463


### Final Questions

1. Which model had the best balanced accuracy score?

    #### Easy Ensemble Classifier Balanced Accuracy Score 0.9254565671948463

2. Which model had the best recall score?

    #### Easy Ensemble Classifier has the best Recall Score high_risk 0.91 and low_risk 0.94

3. Which model had the best geometric mean score?

    #### Easy Ensemble Classifier has the best Geometric Mean high_risk 0.93 and low_risk 0.93

4. What are the top three features?

    #### Balance Random Forest Classifier model's feature_importances_ attribute gives us these 3 top features
    - total_rec_prncp    feature_importance 0.071484
    
    - total_pymnt    feature_importance 0.060788
    
    - total_rec_int    feature_importance 0.059011